Please install nnUNet from https://github.com/rixez/Brats21_KAIST_MRI_Lab, and run this notebook using the nnUNet conda/virtual env. 

### Firstly setting up environment variables

Add your path here instead and copy paste in terminal in your nnUNet Environment:
```
export nnUNet_raw_data_base="/location/to/nnUNet_raw_data_base"
export nnUNet_preprocessed="/location/to/nnUNet_preprocessed"
export RESULTS_FOLDER="/location/to/nnUNet_trained_models"
```

## Brats 2021 nnUNet inference


Make sure your data is in the form of:

`'modalities': {0: 'T1', 1: 'T1ce', 2: 'T2', 3: 'FLAIR'}`

For example:

```
1. BraTS2021_00621_0000.nii.gz
2. BraTS2021_00621_0001.nii.gz
3. BraTS2021_00621_0002.nii.gz
4. BraTS2021_00621_0003.nii.gz

5. BraTS2021_00622_0000.nii.gz
6. BraTS2021_00622_0001.nii.gz
7. BraTS2021_00622_0002.nii.gz
8. BraTS2021_00622_0003.nii.gz
```

Download the model from `https://drive.google.com/file/d/1yWgD1JlEocXRWVMAYOa7YKtQLEhDjhIx/view?usp=sharing`. Extract the model from the rar file and open the directory and go into `3d_fullres`. 

copy that and paste into `/location/to/nnUNet_trained_models/nnUNet/3d_fullres/`

so now it should be 

`/location/to/nnUNet_trained_models/nnUNet/3d_fullres/Task500_BraTS2021`


Basically you are placing the 3d inference model where all the model should be.

### Step 1: Transferring of Data

In [1]:
import os 
from natsort import natsorted
from glob2 import glob

In [10]:
def nnUnet_transfer_images(patients_dir, list_of_modalities, output_dir, exp_name):
    all_patients_dir = natsorted(glob(patients_dir + '/*'))
    
    for i, p in enumerate(all_patients_dir):
        for j, modality in enumerate(list_of_modalities):
            src_file = p + '/' + p.split('/')[-1] + '_' + modality + '.nii.gz'
            dst_file = output_dir + '/' + exp_name + '_' + '{:04}'.format(i+1) + '_000' + str(j) + '.nii.gz' 

            #print(dst_file)
            os.system('cp -r '+ src_file + ' ' + dst_file)    
            
def nnUnet_transfer_segs(patients_dir, output_dir, exp_name):
    
    all_patients_dir = natsorted(glob(patients_dir + '/*'))
    
    for i, p in enumerate(all_patients_dir):
        src_file = p + '/' + p.split('/')[-1] + '_seg.nii.gz'
        dst_file = output_dir + '/' + exp_name + '_'  + '{:04}'.format(i+1) + '.nii.gz'

        #print(dst_file)
        os.system('cp -r '+ src_file + ' ' + dst_file)  

In [11]:
os.system('mkdir /location/to/Task500_BraTS2021')
os.system('mkdir /location/to/Task500_BraTS2021/imagesTr')
os.system('mkdir /location/to/Task500_BraTS2021/labelsTr')
os.system('mkdir /location/to/Task500_BraTS2021/imagesTs')

0

#### Transfer images: (To get into format)

In [12]:
nnUnet_transfer_images(patients_dir = '/media/data/BraTS2021/', 
                     list_of_modalities = ['t1','t1ce','t2','flair'], 
                     output_dir = '/location/to/Task500_BraTS2021/imagesTr', 
                     exp_name = 'BraTS2021')

#### Transfer segs: (To get into format)

In [13]:
nnUnet_transfer_segs(patients_dir = '/media/data/BraTS2021/', 
                     output_dir = '/location/to/Task500_BraTS2021/labelsTr', 
                     exp_name = 'BraTS2021')

#### Creating Dataset JSON (Incase of Training)

In [14]:
from typing import Tuple
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *

In [15]:
def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-12] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques


def generate_dataset_json(output_file: str, imagesTr_dir: str, imagesTs_dir: str, modalities: Tuple,
                          labels: dict, dataset_name: str, license: str = "hands off!", dataset_description: str = "",
                          dataset_reference="", dataset_release='0.0'):
    """
    :param output_file: This needs to be the full path to the dataset.json you intend to write, so
    output_file='DATASET_PATH/dataset.json' where the folder DATASET_PATH points to is the one with the
    imagesTr and labelsTr subfolders
    :param imagesTr_dir: path to the imagesTr folder of that dataset
    :param imagesTs_dir: path to the imagesTs folder of that dataset. Can be None
    :param modalities: tuple of strings with modality names. must be in the same order as the images (first entry
    corresponds to _0000.nii.gz, etc). Example: ('T1', 'T2', 'FLAIR').
    :param labels: dict with int->str (key->value) mapping the label IDs to label names. Note that 0 is always
    supposed to be background! Example: {0: 'background', 1: 'edema', 2: 'enhancing tumor'}
    :param dataset_name: The name of the dataset. Can be anything you want
    :param license:
    :param dataset_description:
    :param dataset_reference: website of the dataset, if available
    :param dataset_release:
    :return:
    """
    train_identifiers = get_identifiers_from_splitted_files(imagesTr_dir)

    if imagesTs_dir is not None:
        test_identifiers = get_identifiers_from_splitted_files(imagesTs_dir)
    else:
        test_identifiers = []

    json_dict = {}
    json_dict['name'] = dataset_name
    json_dict['description'] = dataset_description
    json_dict['tensorImageSize'] = "4D"
    json_dict['reference'] = dataset_reference
    json_dict['licence'] = license
    json_dict['release'] = dataset_release
    json_dict['modality'] = {str(i): modalities[i] for i in range(len(modalities))}
    json_dict['labels'] = {str(i): labels[i] for i in labels.keys()}

    json_dict['numTraining'] = len(train_identifiers)
    json_dict['numTest'] = len(test_identifiers)
    json_dict['training'] = [
        {'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i
        in
        train_identifiers]
    json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_identifiers]

    if not output_file.endswith("dataset.json"):
        print("WARNING: output file name is not dataset.json! This may be intentional or not. You decide. "
              "Proceeding anyways...")
    save_json(json_dict, os.path.join(output_file))

In [17]:
generate_dataset_json(
    output_file = "/location/to/Task500_BraTS2021/dataset.json",
    imagesTr_dir = "/location/to/Task500_BraTS2021/imagesTr",
    imagesTs_dir = "/location/to/Task500_BraTS2021/imagesTs",
    modalities = ('t1', 't1ce', 't2', 'flair'),
    labels = {
        "0": "background",
        "1": "Class 1",
        "2": "Class 2",
        "3": "Class 3"
    },
    dataset_name = "Brats Training",
    license = "hands off",
    dataset_description = "nnUnet experiment for Brats2021",
    dataset_reference = "",
    dataset_release = "1.0"
)

### Step 2: Try Inference

You can transfer some images in a folder to try inference

Using Model 1
```
nnUNet_predict -i <input_folder> -o <output_folder1> -t 500 -m 3d_fullres -tr nnUNetTrainerV2BraTSRegions_DA4_BN_BD --save_npz
```
Using Model 2
```
nnUNet_predict -i <input_folder> -o <output_folder2> -t 500 -m 3d_fullres -tr nnUNetTrainerV2BraTSRegions_DA4_BN_BD_largeUnet_Groupnorm --save_npz
```

Using Ensemble Model
```
nnUNet_ensemble -f <output_folder1> <output_folder2> -o <final_output_folder>
```

### Step 3: Fine-tuning the model


Similar to the Transfer learning Notebook I sent

#### Preprocessing:

Example:

A - 
```
nnUNet_plan_and_preprocess -t 501 -pl3d ExperimentPlanner3D_v21_Pretrained -overwrite_plans /media/rnd/nnUNet/RESULTS_FOLDER/nnUNet/3d_fullres/Task500_BraTS2021/nnUNetTrainerV2BraTSRegions_DA4_BN_BD_largeUnet_Groupnorm__nnUNetPlansv2.1/plans.pkl -overwrite_plans_identifier GROUP_NORM_FT
```

B - 

```
nnUNet_plan_and_preprocess -t 501 -pl3d ExperimentPlanner3D_v21_Pretrained -overwrite_plans /media/rnd/nnUNet/RESULTS_FOLDER/nnUNet/3d_fullres/Task500_BraTS2021/nnUNetTrainerV2BraTSRegions_DA4_BN_BD__nnUNetPlansv2.1/plans.pkl -overwrite_plans_identifier OTHER_FT
```

#### Training:

And now that we have processed the data into the version of the model we need to fine tune. We just need to train. Which is a little different but similar:

```
CUDA_VISIBLE_DEVICES=0 nnUNet_train 3d_fullres nnUNetTrainerV2 Task501_BraTS2021 all -p nnUNetPlans_pretrained_GROUP_NORM_FT --npz -pretrained_weights /media/rnd/nnUNet/RESULTS_FOLDER/nnUNet/3d_fullres/Task500_BraTS2021/nnUNetTrainerV2BraTSRegions_DA4_BN_BD_largeUnet_Groupnorm__nnUNetPlansv2.1/fold_0/model_final_checkpoint.model
```